In [32]:
import numpy as np
from collections import defaultdict
import sys
import time
import random 


class Member:    # a document: tfidf representation, label, name
    def __init__(self, r_d, label, doc_id):
        self._r_d = r_d    # tfidf representation
        self._label = label
        self._doc_id = doc_id
    
class Cluster:   # contains its centroid and its members
    def __init__(self):
        self._centroid = None
        self._members = []
    def reset_members(self):  # after recalculating centroids => clear all clusters to assign again
        self._members = []
    def add_member(self, member):
        self._members.append(member)
    
class K_Means:
    def __init__(self, num_clusters):
        self._num_clusters = num_clusters
        self._clusters = [Cluster() for _ in range(self._num_clusters)]
        self._E = []   # list of current centroids of K clusters
        self._S = 0     # current total similarity
        
    def load_data(self, path):
        def sparse_to_dense(sparsed_r_d, vocab_size):
            r_d = np.zeros(vocab_size)
            split_sparse_r_d = sparsed_r_d.split()
            for item in split_sparse_r_d:
                idx = int(item.split(':')[0])
                tfidf = float(item.split(':')[1])
                r_d[idx] = tfidf
            return r_d    # numpy array
        
        with open("D:\\movedFromC\\123\\20192\\PRJ2\\Project2\\words_idfs.txt") as f:
            vocab_size = len(f.read().splitlines())
            
        with open(path) as f:
            doc_lines = f.read().splitlines()
        self._data = []       # one doc (a member) is an item of data
        self._label_count = defaultdict(int)    # count number of occurences of one class
        for data_id, data in enumerate(doc_lines):
            features = data.split('<fff>')
            label = int(features[0])
            doc_id = int(features[1])
            self._label_count[label] += 1
            r_d = sparse_to_dense(features[2], vocab_size)
            
            self._data.append(Member(r_d, label, doc_id))
        
    def compute_similarity(self, member, centroid):   
        # choosing Euclidean distance
        dist = np.linalg.norm(member._r_d - centroid) + 1e-12
        return 1. / dist

    def select_cluster_for(self, member):
        best_cluster = None
        max_similarity = -1
        for cluster in self._clusters:
            similarity = self.compute_similarity(member, cluster._centroid)
            if similarity > max_similarity:
                best_cluster = cluster
                best_similarity = similarity
        best_cluster.add_member(member)      
        return max_similarity      # similarity between member and its found cluster
    
    def update_centroids_for(self, cluster):
        # after assigning data points to theirs clusters => update centroids
        # one member => one vector of tfidf => compute mean vector 
        # in order to normalize, find the square of the sum of (each element)^2 from the mean vector
    
        member_r_ds = [member._r_d for member in cluster._members]   # get tfidf
        aver_r_d = np.mean(member_r_ds, axis = 0)  # compute mean => new_centroid
        denom = np.sqrt(np.sum(aver_r_d ** 2))
       # new_centroid = np.array([numerator / denom for numerator in aver_r_d])  # normalize new_centroid
        new_centroid = aver_r_d / denom
        cluster._centroid = new_centroid
        
    def init_centroids(self, init_strategy):
        types = ['k-means++', 'random']
        assert init_strategy in types
        
        if init_strategy == 'k-means++':
            # randomly pick one data point as the first centroid
            self._clusters[0]._centroid = self._data[random.randrange(0, len(self._data)-1), 1]._r_d
            idx = 0
            d = sys.maxsize
            # a list in which each item is the distance between a data point and the nearest previously chosen centroid
            dist_points_centroid = [d] * len(self._data)
            dist_points_centroid = np.array(dist_points_centroid)
            
            # loop to find the rest K-1 centroids
            for centroid_idx in range(self._num_clusters - 1):
                for i in range(len(self._data)):
                    point = self._data[i]._r_d
                    # only need to compute distance between the point and the newest centroid
                    temp_dist = np.sum((self._clusters[idx] - point)**2)
                    dist_points_centroid[i] = min(dist_points_centroid[i], temp_dist)
                
                # choose the data point that has the largest distance with its nearest centroid
                next_centroid = self._data[np.argmax(dist_points_centroid)]._r_d
                self._clusters[idx+1]._centroid = next_centroid
                idx += 1
                
        if init_strategy == 'random':
            randomed = []
            for i in range(self._num_clusters):
                data_idx = random.randrange(0, len(self._data)-1)
                while (data_idx in randomed):
                    data_idx = random.randrange(0, len(self._data)-1)
                self._clusters[i]._centroid = self._data[data_idx]._r_d
                randomed.append(data_idx)
            
        
    def stopping_condition(self, criterion, threshold):
        criteria = ['centroid', 'similarity', 'max_iters']
        assert criterion in criteria
        if criterion == 'max_iters':
            if self._iterations >= threshold:
                return True
            else: 
                return False
        elif criterion == 'centroid':    # |E_new \ E| < threshold
            E_new = [list(cluster._centroid) for cluster in self._clusters]
            E_new_minus_E = [centroid for centroid in E_new if centroid not in self._E]
            self._E = E_new
            if (len(E_new_minus_E) < threshold):
                return True
            else:
                return False
        else: 
            S_new_minus_S = self._new_S - self._S
            self._S = self._new_S
            if S_new_minus_S <= threshold:
                return True
            else:
                return False
            
    def run(self, init_strategy, criterion, threshold):
        #init centroids
        self.init_centroids(init_strategy)
        
        self._iterations = 0
        self._new_S = 0
        
        while True:
            print("Epoch ", str(self._iterations+1))
            # clear all members in each cluster
            for cluster in self._clusters:
                cluster.reset_members()
            # select cluster for every data point
            for member in self._data:
                max_similarity = self.select_cluster_for(member)
                self._new_S += max_similarity
            # update centroids for clusters:
            for cluster in self._clusters:
                self.update_centroids_for(cluster)
            
            #check stopping_conddition:
            self._iterations += 1;
            if self.stopping_condition(criterion, threshold):
                break
        
        
    def compute_purity(self):
        numerator = 0
        for cluster in self._clusters:
            members_labels = [member._label for member in cluster._members]   # list of labels of all doc in one cluster
            max_count = max([members_labels.count(label) for label in range(20)]) # find the number of times the class that appears the most in that cluster
            numerator += max_count
        return numerator * 1. / len(self._data)
    
    def comupte_NMI(self):
        I, H_omega, H_c, N = 0., 0., 0., len(self._data)
        for label in range(20):
            # compute H_c
            cj = self._label_count[label]
            H_c += -(cj/N * np.log10(cj/N))
        for cluster in self._clusters:
            # compute H_omage
            wk = len(cluster._members) * 1.
            H_omega += -(wk/N * np.log10(wk/N))
            
            # compute I
            member_labels = [member._label for member in cluster._members]
            for label in range(20):
                wk_cj = member_labels.count(label) * 1.
                cj = self._label_count[label]
                I += (wk_cj / N) * np.log10(N * wk_cj / (wk * cj) + 1e-12)
        
        return I * 2. / (H_c + H_omega)

In [33]:
num_clusters = 20
km = K_Means(num_clusters)

km.load_data("D:\\movedFromC\\123\\20192\\PRJ2\\Project2\\words_tfidfs.txt")
            

In [6]:
#len(km._data)

11314

In [ ]:
import time

best_model = km
tmp_training_purity = 0
best_training_purity  = 0

num_attempts = 5
init_strategy = "random"
criterion = "max_iters"
threshold = 25

for i in range(num_attempts):
    t1 = time.time()
    km.run(init_strategy, criterion, threshold)
    t2 = time.time()
    print("Training time: ", t2-t1)
    tmp_training_purity = km.compute_purity()
    if (tmp_training_purity > best_training_purity):
        best_model = km
    print("Purity: ", tmp_training_purity)
    print("-------------------")

In [ ]:
print("Best model")
print("Purity: ", km.compute_purity())
print("NMI: ", km.compute_NMI())